# Combined Submission Notebook
**Project:** Portfolio Optimization with CVaR (Linear Programming)  
**Repo:** `AHMerrill/optimization_1`  
**Notebook:** `combined_submission.ipynb`

> Use this single notebook to run end-to-end and export figures/tables for the PDF report.


## Table of Contents
1. Part 1 — Data preparation & return calculation  
2. Part 2 — Portfolio optimization (β=0.95, in-sample vs out-of-sample)  
3. Part 3 — Sensitivity to β (0.90, 0.95, 0.99)  
4. Part 4 — Conservative min–max monthly CVaR (2019)  
5. Part 5 — Monthly re-optimization (2020 rolling windows)  
6. Part 6 — Stability check of monthly portfolios  
7. Part 7 — Final report assembly (figures & summary)


In [ ]:
# === Setup & Imports ===
import os, sys, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: uncomment if you plan to use these
# import gurobipy as gp
# from gurobipy import GRB
# import cvxpy as cp

plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['figure.dpi'] = 120

# Paths (edit as needed)
DATA_2019 = "prices_2019.csv"   # replace with your actual path
DATA_2020 = "prices_2020.csv"   # replace with your actual path

OUTPUT_DIR = "artifacts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Environment ready. Edit DATA_2019/DATA_2020 if needed.")


## Part 1 — Data preparation & return calculation

In [ ]:
# TODO: Load price CSVs, compute daily percentage returns, split NDX vs investable universe
# Example skeleton:
# prices_2019 = pd.read_csv(DATA_2019, parse_dates=['Date']).sort_values('Date')
# prices_2020 = pd.read_csv(DATA_2020, parse_dates=['Date']).sort_values('Date')
# rets_2019 = prices_2019.set_index('Date').pct_change().dropna(how='all')
# rets_2020 = prices_2020.set_index('Date').pct_change().dropna(how='all')
# ndx_2019 = rets_2019['NDX']; R19 = rets_2019.drop(columns=['NDX'])
# ndx_2020 = rets_2020['NDX']; R20 = rets_2020.drop(columns=['NDX'])
# display(R19.head()); display(R20.head())


## Part 2 — Portfolio optimization (β=0.95, in-sample vs out-of-sample)

In [ ]:
# TODO: Implement CVaR LP (Gurobi or CVXPY), solve on 2019 at beta=0.95 with return floor R=0.0002
# Evaluate VaR/CVaR in 2019 and 2020 using fixed weights
# Save figures to OUTPUT_DIR


## Part 3 — Sensitivity to β (0.90, 0.95, 0.99)

In [ ]:
# TODO: Re-run solver for beta in [0.90, 0.95, 0.99]. Plot weight bars and compare allocations.
# Save 'weights_by_beta.png' in OUTPUT_DIR


## Part 4 — Conservative min–max monthly CVaR (2019)

In [ ]:
# TODO: Build min-max monthly CVaR model:
#   - One x for the year
#   - alpha_m, u_m per month
#   - minimize t s.t. each month’s CVaR <= t
# Save results/plot: 'minmax_monthly_cvar.png' in OUTPUT_DIR


## Part 5 — Monthly re-optimization (2020 rolling windows)

In [ ]:
# TODO: For each month in 2020, use a 12-month rolling window to optimize x, then evaluate that month.
# Build a DataFrame of month, VaR, CVaR, and save a time series plot to OUTPUT_DIR.


## Part 6 — Stability check of monthly portfolios

In [ ]:
# TODO: Check |x_t - x_{t-1}| <= 0.05 for all assets. Print whether stable.
# If not stable, describe constraints you'd add (documentation cell in Part 7).


## Part 7 — Final report assembly (figures & summary)

In [ ]:
# TODO: Collate key metrics and embed figures.
# Export a concise markdown or HTML summary if desired (optional).
# Example: generate a CSV/markdown file in OUTPUT_DIR summarizing results.
